In [71]:
import pandas as pd
import re

In [72]:
import os
import pickle

file_path = "../data/data_diff_prices/results_sims.pkl"

with open(file_path, 'rb') as file:
    loaded_pickle = pickle.load(file)


In [26]:
loaded_pickle

[           mean_1    mean_2     std_1     std_2
 0        0.262635  0.240252  0.091602  0.101684
 1        0.286715  0.235128  0.064674  0.073625
 2        0.257227  0.312703  0.068247  0.086338
 3        0.284457  0.256298  0.099622  0.058875
 4        0.279988  0.313715  0.086410  0.087416
 ...           ...       ...       ...       ...
 1499995  0.286818  0.260951  0.075007  0.069130
 1499996  0.304691  0.245855  0.069743  0.080765
 1499997  0.305895  0.239457  0.059273  0.077386
 1499998  0.288710  0.270520  0.090006  0.090487
 1499999  0.305856  0.252723  0.075188  0.077491
 
 [1500000 rows x 4 columns],
            mean_1    mean_2     std_1     std_2
 0        0.278873  0.297192  0.090519  0.095268
 1        0.313869  0.261908  0.093980  0.053683
 2        0.273956  0.313428  0.094458  0.078583
 3        0.292893  0.289476  0.102835  0.074873
 4        0.288511  0.303042  0.077816  0.069953
 ...           ...       ...       ...       ...
 1499995  0.341374  0.240362  0.080578

In [27]:
import numpy as np

In [73]:
hyperparam = []
for alpha in np.linspace(0.025, 0.2, 5):
    for beta in np.linspace(1e-06, 2e-05, 5):
        hyperparam.append((alpha, beta))

for i in range(len(loaded_pickle)):
    loaded_pickle[i]['alpha'] = hyperparam[i][0]
    loaded_pickle[i]['beta'] = hyperparam[i][1]
    loaded_pickle[i]['iteration'] = loaded_pickle[i].index
    loaded_pickle[i] = loaded_pickle[i][~((loaded_pickle[i]['iteration']%1000!=999) & (loaded_pickle[i]['iteration']!=0))]

In [74]:
sim_df = pd.concat(loaded_pickle, ignore_index=True)

In [75]:
sim_df = sim_df.set_index(['beta','alpha','iteration'])

In [76]:
sim_df

mean_1    mean_2     std_1     std_2
beta     alpha iteration                                        
0.000001 0.025 0          0.262635  0.240252  0.091602  0.101684
               999        0.256585  0.303286  0.072936  0.081011
               1999       0.313830  0.256590  0.084654  0.091486
               2999       0.278233  0.256782  0.093830  0.079103
               3999       0.284204  0.273233  0.095725  0.085654
...                            ...       ...       ...       ...
0.000020 0.200 1495999    0.357611  0.205213  0.038455  0.046224
               1496999    0.367373  0.216004  0.034857  0.053856
               1497999    0.348520  0.215335  0.048902  0.058518
               1498999    0.354778  0.235582  0.048067  0.080096
               1499999    0.357611  0.205213  0.038455  0.046224

[37525 rows x 4 columns]

In [77]:
sim_df.rename(columns={'mean_1':'Reward1', 'mean_2':'Reward2'}, inplace=True)

In [78]:
sim_df

Reward1   Reward2     std_1     std_2
beta     alpha iteration                                        
0.000001 0.025 0          0.262635  0.240252  0.091602  0.101684
               999        0.256585  0.303286  0.072936  0.081011
               1999       0.313830  0.256590  0.084654  0.091486
               2999       0.278233  0.256782  0.093830  0.079103
               3999       0.284204  0.273233  0.095725  0.085654
...                            ...       ...       ...       ...
0.000020 0.200 1495999    0.357611  0.205213  0.038455  0.046224
               1496999    0.367373  0.216004  0.034857  0.053856
               1497999    0.348520  0.215335  0.048902  0.058518
               1498999    0.354778  0.235582  0.048067  0.080096
               1499999    0.357611  0.205213  0.038455  0.046224

[37525 rows x 4 columns]

In [80]:
sim_df.to_csv('../final_data/res_diff_pricing_time.csv', index=True)

In [3]:
import os
import pickle

folder_path = "../data/data/data_sim/"

# Get a list of all files in the folder
pickle_files = [file for file in os.listdir(folder_path) if file.endswith('.pkl')]

# Loop through the list of pickle files and load them
loaded_pickles = []
for file_name in pickle_files:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'rb') as file:
        loaded_pickle = pickle.load(file)
        loaded_pickles.append(loaded_pickle)

# Now, loaded_pickles contains all the pickles from the folder
sim_df = pd.concat(loaded_pickles, ignore_index=True)

In [4]:
sim_df.dtypes

Iteration     object
Rewards       object
Prices        object
Epsilon       object
beta         float64
index          int64
dtype: object

In [5]:
#Convert into int if digit
sim_df['Iteration'] = sim_df['Iteration'].apply(lambda x: int(x) if str(x).isdigit() else x)


#Separate the profits, prices and Epsilon of each player
sim_df[['Reward1', 'Reward2']] = sim_df['Rewards'].apply(lambda x: pd.Series(x))
sim_df[['Price1', 'Price2']] = sim_df['Prices'].apply(lambda x: pd.Series(x))
sim_df[['Epsilon1', 'Epsilon2']] = sim_df['Epsilon'].apply(lambda x: pd.Series(x))

#Drop the useless columns
sim_df = sim_df.drop(columns=['Rewards', 'Prices', 'Epsilon'])

/var/folders/1v/lw80nwsd61d7sp0q_pzj0ppm0000gp/T/ipykernel_996/3638808642.py:6: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  sim_df[['Reward1', 'Reward2']] = sim_df['Rewards'].apply(lambda x: pd.Series(x))
/var/folders/1v/lw80nwsd61d7sp0q_pzj0ppm0000gp/T/ipykernel_996/3638808642.py:7: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  sim_df[['Price1', 'Price2']] = sim_df['Prices'].apply(lambda x: pd.Series(x))
/var/folders/1v/lw80nwsd61d7sp0q_pzj0ppm0000gp/T/ipykernel_996/3638808642.py:8: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  sim_df[['Epsilon1', 'Epsilon2']] = sim_df['Epsilon'].apply(lambda x: pd.Series(x))


In [6]:
sim_df

,Iteration,beta,index,Reward1,Reward2,Price1,Price2,Epsilon1,Epsilon2
0,0,0.002500,1,0.200833,0.280442,1.61950,1.46350,1.000000e+00,1.000000e+00
1,10000,0.002500,1,0.191614,0.294219,1.65850,1.46350,1.392271e-11,1.392271e-11
2,20000,0.002500,1,0.191614,0.294219,1.65850,1.46350,1.933578e-22,1.933578e-22
3,30000,0.002500,1,0.191614,0.294219,1.65850,1.46350,2.685342e-33,2.685342e-33
4,40000,0.002500,1,0.191614,0.294219,1.65850,1.46350,3.729388e-44,3.729388e-44
...,...,...,...,...,...,...,...,...,...
200395,9970000,0.000025,10,0.169348,0.374261,1.85350,1.54150,5.650817e-109,5.650817e-109
200396,9980000,0.000025,10,0.292024,0.362491,1.85350,1.77550,4.400861e-109,4.400861e-109
200397,9990000,0.000025,10,0.372947,0.297938,1.85350,1.93150,3.427394e-109,3.427394e-109
200398,10000000,0.000025,10,0.228329,0.384294,1.85350,1.65850,2.669257e-109,2.669257e-109


In [7]:
sim_df.to_csv('../final_data/res_same_beta.csv', index=False)

In [8]:
import os
import pickle

folder_path = "../data/data/data_Q/"

# Get a list of all files in the folder
pickle_files = [file for file in os.listdir(folder_path) if file.endswith('.pkl')]

# Loop through the list of pickle files and load them
loaded_pickles = []
for file_name in pickle_files:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'rb') as file:
        loaded_pickle = pickle.load(file)
        loaded_pickles.append(loaded_pickle)

# Now, loaded_pickles contains all the pickles from the folder
Q_df = pd.concat(loaded_pickles, ignore_index=True)

In [9]:
Q_df.to_csv('../final_data/Q_same_beta.csv', index=False)

In [10]:
import os
import pickle

folder_path = "../data/data_simbis/"

# Get a list of all files in the folder
pickle_files = [file for file in os.listdir(folder_path) if file.endswith('.pkl')]

# Loop through the list of pickle files and load them
loaded_pickles = []
for file_name in pickle_files:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'rb') as file:
        loaded_pickle = pickle.load(file)
        loaded_pickles.append(loaded_pickle)

# Now, loaded_pickles contains all the pickles from the folder
sim_df = pd.concat(loaded_pickles, ignore_index=True)

In [11]:
sim_df.dtypes

Iteration     object
Rewards       object
Prices        object
Epsilon       object
beta         float64
index          int64
dtype: object

In [12]:
sim_df['Rewards'][0][1]

0.28044202997287715

In [13]:
def extract_numbers(text):
    numbers = re.findall(r'-?\d+\.?\d*(?:e[-+]?\d+)?', text)
    return [float(num) for num in numbers]

# # Apply the function to the column and convert the strings into lists of two numbers
# sim_df['Rewards'] = sim_df['Rewards'].apply(extract_numbers)
# sim_df['Prices'] = sim_df['Prices'].apply(extract_numbers)
# sim_df['Epsilon'] = sim_df['Epsilon'].apply(extract_numbers)

#Convert into int if digit
sim_df['Iteration'] = sim_df['Iteration'].apply(lambda x: int(x) if str(x).isdigit() else x)


#Separate the profits, prices and Epsilon of each player
sim_df[['Reward1', 'Reward2']] = sim_df['Rewards'].apply(lambda x: pd.Series(x))
sim_df[['Price1', 'Price2']] = sim_df['Prices'].apply(lambda x: pd.Series(x))
sim_df[['Epsilon1', 'Epsilon2']] = sim_df['Epsilon'].apply(lambda x: pd.Series(x))

#Drop the useless columns
sim_df = sim_df.drop(columns=['Rewards', 'Prices', 'Epsilon'])

/var/folders/1v/lw80nwsd61d7sp0q_pzj0ppm0000gp/T/ipykernel_996/355113774.py:15: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  sim_df[['Reward1', 'Reward2']] = sim_df['Rewards'].apply(lambda x: pd.Series(x))
/var/folders/1v/lw80nwsd61d7sp0q_pzj0ppm0000gp/T/ipykernel_996/355113774.py:16: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  sim_df[['Price1', 'Price2']] = sim_df['Prices'].apply(lambda x: pd.Series(x))
/var/folders/1v/lw80nwsd61d7sp0q_pzj0ppm0000gp/T/ipykernel_996/355113774.py:17: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  sim_df[['Epsilon1', 'Epsilon2']] = sim_df['Epsilon'].apply(lambda x: pd.Series(x))


In [14]:
sim_df

,Iteration,beta,index,Reward1,Reward2,Price1,Price2,Epsilon1,Epsilon2
0,0,0.002500,1,0.200833,0.280442,1.6195,1.4635,1.000000e+00,1.000000e+00
1,10000,0.002500,1,0.217067,0.202776,1.4245,1.4635,1.392271e-11,9.048465e-01
2,20000,0.002500,1,0.298722,0.271664,1.6195,1.6585,1.933578e-22,8.187389e-01
3,30000,0.002500,1,0.207851,0.384671,1.8535,1.6195,2.685342e-33,7.408256e-01
4,40000,0.002500,1,0.212486,0.399740,1.8925,1.6585,3.729388e-44,6.703267e-01
...,...,...,...,...,...,...,...,...,...
200395,9970000,0.000025,10,0.159728,0.405275,1.9315,1.5805,5.650817e-109,5.021628e-44
200396,9980000,0.000025,10,0.159728,0.405275,1.9315,1.5805,4.400861e-109,4.543756e-44
200397,9990000,0.000025,10,0.159728,0.405275,1.9315,1.5805,3.427394e-109,4.111361e-44
200398,10000000,0.000025,10,0.159728,0.405275,1.9315,1.5805,2.669257e-109,3.720113e-44


In [15]:
sim_df.to_csv('../final_data/res_diff_beta.csv', index=False)

In [16]:
import os
import pickle

folder_path = "../data/data_Qbis/"

# Get a list of all files in the folder
pickle_files = [file for file in os.listdir(folder_path) if file.endswith('.pkl')]

# Loop through the list of pickle files and load them
loaded_pickles = []
for file_name in pickle_files:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'rb') as file:
        loaded_pickle = pickle.load(file)
        loaded_pickles.append(loaded_pickle)

# Now, loaded_pickles contains all the pickles from the folder
Q_df = pd.concat(loaded_pickles, ignore_index=True)

In [17]:
Q_df.to_csv('../final_data/Q_diff_beta.csv', index=False)